In [31]:
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, ELU
from keras.activations import relu, softmax
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
from keras.regularizers import l2
import math
import os
import json

def read_data():
    data_path = "driving_log.csv"
    columns = ['center_image','left_image','right_image','angle','throttle','speed']
    df = pd.read_csv(data_path, header = None, names = columns)
    
    center_images = df.center_image.tolist()
    angle = df.angle.tolist()
    left_images = df.left_image.tolist()
    right_images = df.right_image.tolist()
    
    return (center_images, angle, left_images, right_images)
    
    
        
def shuffle(center_images, angle):
    
    return shuffle(center_images, angle)





def get_model(time_len=1):
    #ch, row, col = 3, 160, 320  # camera format
    ch, row, col = 160, 320, 3

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(ch, row, col),
            output_shape=(ch, row, col)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same",W_regularizer = l2(0.001)))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same",W_regularizer = l2(0.001)))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same",W_regularizer = l2(0.001)))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")

    return model


def flip(image, angle):
  new_image = cv2.flip(image,1)
  new_angle = angle*(-1)
  return new_image, new_angle


def train_generator(batch_size, center_list, angle_list):
    
    while 1:
        train_center = np.zeros((batch_size,160, 320, 3), dtype = np.float32)
        train_angle = np.zeros((batch_size,), dtype = np.float32)
        
        
        for i in range(batch_size):
            
            rand = int(np.random.choice(len(center_list),1))
         
            train_center[i] = mpimg.imread(center_list[i].lstrip())
            train_angle[i] = angle_list[i]
            
            flip_coin = random.randint(0,1)
            
            if flip_coin == 1:
                train_center[i], train_angle[i] = flip(train_center[i], train_angle[i])
        
        yield train_center, train_angle
            


center, angle, left, right = read_data()


center_train, center_valid, angle_train, angle_valid = train_test_split(center, angle, test_size = 0.20, random_state = 100) 


d_straight, d_left, d_right = [], [], []
a_straight, a_left, a_right = [], [], []

#for i in (0,len(center)-1):
#    d_right.append(right[i])
#    a_right.append(angle[i]-0.25)
#    d_left.append(left[i])
#a_left.append(angle[i]+0.25)

final_train_image = center_train+d_right+d_left
final_train_angle = angle_train+a_right+a_left

batch_size = 128
train_gen = train_generator(batch_size, final_train_image, final_train_angle)
valid_gen = train_generator(batch_size, center_valid, angle_valid)



model = get_model()
model.fit_generator(
train_gen,
samples_per_epoch=math.ceil(len(center_train)),
nb_epoch=5,
validation_data=valid_gen,
nb_val_samples=len(center_valid)
)
print("Saving model weights and configuration file.")


model.save_weights("model.h5")
with open('model.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)


    
  

Epoch 1/5
6400/6429 [============================>.] - ETA: 0s - loss: 0.9504

/Users/ying/anaconda3/envs/CarND-TensorFlow-Lab/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6528/6429 [==============================] - 113s - loss: 0.9343 - val_loss: 0.2552
Epoch 2/5
6528/6429 [==============================] - 111s - loss: 0.1180 - val_loss: 0.2421
Epoch 3/5
6528/6429 [==============================] - 96s - loss: 0.1059 - val_loss: 0.2283
Epoch 4/5
6528/6429 [==============================] - 96s - loss: 0.0981 - val_loss: 0.2154
Epoch 5/5
6528/6429 [==============================] - 99s - loss: 0.0942 - val_loss: 0.2096
Saving model weights and configuration file.
